In [50]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

In [51]:
URL = 'https://books.toscrape.com/catalogue/page-'
books,price,ratings,stock=[],[],[],[]

In [52]:
for page in range(1,51):
    src=requests.get(URL + str(page) + '.html')
    Scrape = BeautifulSoup(src.text,'html.parser')
    
    for article in Scrape.find_all('article'):
        # Grabbing book titles
        title = article.h3.a.text
        books.append(title)

        # Grabbing book price
        cost = article.find('div', class_='product_price').p.text
        price.append(cost)

         # Grabbing book rating
        rating_tag = article.find('p', class_='star-rating')
        rating_classes = rating_tag['class']  # Get all classes of the rating tag
        rating = rating_classes[1]  # The second class represents the rating
        ratings.append(rating)

        # Grabbing stock availability of each book
        status = article.find('p', class_='instock availability').text.strip()
        stock.append(status)

In [53]:
Scraped_Data = {'BOOK_TITLE':books,'PRICE':price,'RATING':ratings,'AVAILABILITY':stock}
data_scraped = pd.DataFrame(Scraped_Data)
data_scraped.head()

,BOOK_TITLE,PRICE,RATING,AVAILABILITY
0,A Light in the ...,Â£51.77,Three,In stock
1,Tipping the Velvet,Â£53.74,One,In stock
2,Soumission,Â£50.10,One,In stock
3,Sharp Objects,Â£47.82,Four,In stock
4,Sapiens: A Brief History ...,Â£54.23,Five,In stock


In [54]:
data_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BOOK_TITLE    1000 non-null   object
 1   PRICE         1000 non-null   object
 2   RATING        1000 non-null   object
 3   AVAILABILITY  1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [55]:
data_scraped.nunique()

BOOK_TITLE      992
PRICE           903
RATING            5
AVAILABILITY      1
dtype: int64

In [56]:
data_scraped['RATING'].unique()

array(['Three', 'One', 'Four', 'Five', 'Two'], dtype=object)

In [57]:
data_scraped['AVAILABILITY'].unique()

array(['In stock'], dtype=object)

In [58]:
df=data_scraped.copy()

In [29]:
# src=requests.get(URL + str(1) + '.html')
# Scrape = BeautifulSoup(Source.text,'html.parser')
# for article in Scrape.find_all('article'):
#     print(article)
#     print('--------------------------------------------------------------')

### Transformations

In [59]:
#removing "Â£" in the price column
df['PRICE'] = df['PRICE'].str.replace('Â£', '')
df['PRICE'] = pd.to_numeric(df['PRICE'])
df.rename(columns={'PRICE': 'PRICE_IN_EUROS'}, inplace=True)

#transforming rating
rating_mapping = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
df['RATING'] = df['RATING'].map(rating_mapping)

In [60]:
df

,BOOK_TITLE,PRICE_IN_EUROS,RATING,AVAILABILITY
0,A Light in the ...,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.10,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History ...,54.23,5,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's ...,55.53,1,In stock
996,"Ajin: Demi-Human, Volume 1 ...",57.06,4,In stock
997,A Spy's Devotion (The ...,16.97,5,In stock
998,1st to Die (Women's ...,53.98,1,In stock


In [63]:
df.to_csv("BS_SCRAPED_DATA.csv",index=False)

In [33]:
import snowflake.connector as sf

In [34]:
conn = sf.connect(
    user='XXXXXX',
    password='XXXXXX',
    account='XXXXXXX',
    warehouse='COMPUTE_WH',
    database='CODING_CHALLENGE',
    schema='PUBLIC',
    role='ACCOUNTADMIN'
    )

In [35]:
cur=conn.cursor()

In [64]:
df.columns

Index(['BOOK_TITLE', 'PRICE_IN_EUROS', 'RATING', 'AVAILABILITY'], dtype='object')

In [37]:
# Specify the table name
table_name = 'book_store'

# Define the CREATE TABLE SQL query
create_table_query = f'''
CREATE TABLE {table_name} (
    book_title VARCHAR,
    price_in_euros FLOAT,
    rating INT,
    availability VARCHAR
)
'''

cur.execute(create_table_query)

In [38]:
from snowflake.connector.pandas_tools import write_pandas

In [65]:
write_pandas(conn,df,table_name="BOOK_STORE")

(True,
 1,
 1000,
 [('fjsjlzhlqp/file0.txt',
   'LOADED',
   1000,
   1000,
   1,
   0,
   None,
   None,
   None,
   None)])

Hence the table is successfully loaded into snowflake